In [ ]:
# 🏥 ANÁLISIS DE COSTOS DE SEGURO MÉDICO (LISTO PARA REGRESIÓN)
# Elaborado por: Amy Frías

# ======================================================
# 🔹 1. Importación de librerías
# ======================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from google.colab import files  # Solo si usas Colab

# ======================================================
# 🔹 2. Cargar el dataset subido por el usuario
# ======================================================
uploaded = files.upload()  # Abrirá el diálogo para subir CSV
file_name = list(uploaded.keys())[0]
# Fix: Use semicolon as delimiter and specify encoding
data = pd.read_csv(file_name, delimiter=';', encoding='utf-8')

# Vista rápida del dataset
print("Primeras filas del dataset:")
print(data.head())
print("\nInformación general del dataset:")
print(data.info())
print("\nDescripción estadística del dataset:")
print(data.describe())

# ======================================================
# 🔹 3. Exploración y limpieza de datos
# ======================================================
# Verificar valores faltantes
print("\nValores faltantes por columna:")
print(data.isnull().sum())

# Convertir variables categóricas a tipo 'category'
# Fix: Use correct column names
categorical_cols = ['sex', 'smoker', 'region']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# ======================================================
# 🔹 4. Análisis univariante
# ======================================================
print("\n--- Análisis univariante ---")
# Variables numéricas
num_cols = ['age', 'bmi', 'children', 'charges']
for col in num_cols:
    plt.figure(figsize=(6,4))
    sns.histplot(data[col], kde=True, bins=30, color='skyblue')
    plt.title(f'Distribución de {col}')
    plt.show()
    print(data[col].describe())

# Variables categóricas
for col in categorical_cols:
    plt.figure(figsize=(6,4))
    sns.countplot(x=col, data=data, palette='pastel')
    plt.title(f'Conteo por {col}')
    plt.show()
    print(data[col].value_counts())

# ======================================================
# 🔹 5. Filtrado de outliers en 'cargos'
# ======================================================
Q1 = data['charges'].quantile(0.25)
Q3 = data['charges'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5*IQR
upper = Q3 + 1.5*IQR

data_filtered = data[(data['charges'] >= lower) & (data['charges'] <= upper)]
print(f"\nFiltrado realizado. Filas originales: {len(data)}, Filas filtradas: {len(data_filtered)}")

# ======================================================
# 🔹 6. Transformación de variables categóricas en dummies
# ======================================================
data_ready = pd.get_dummies(data_filtered, columns=categorical_cols, drop_first=True)
print("\nDataset listo para regresión (variables dummies creadas):")
print(data_ready.head())
print(data_ready.info())

# ======================================================
# 🔹 7. Análisis bivariante (scatter y boxplot)
# ======================================================
# Variables numéricas vs cargos
for col in ['age', 'bmi', 'children']:
    plt.figure(figsize=(6,4))
    sns.scatterplot(x=col, y='charges', data=data_ready, color='purple')
    plt.title(f'{col} vs Cargos')
    plt.show()

# Variables categóricas vs cargos (dummies)
dummy_cols = [col for col in data_ready.columns if '_' in col]
for col in dummy_cols:
    plt.figure(figsize=(6,4))
    sns.boxplot(x=col, y='charges', data=data_ready, palette='pastel')
    plt.title(f'{col} vs Cargos')
    plt.show()

# ======================================================
# 🔹 8. Matriz de correlación
# ======================================================
plt.figure(figsize=(12,10))
corr = data_ready.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Matriz de correlación')
plt.show()

# ======================================================
# 🔹 9. Correlación ordenada respecto a 'cargos'
# ======================================================
corr_target = corr['charges'].sort_values(ascending=False)
print("\nCorrelación de todas las variables con 'cargos':")
print(corr_target)

# Gráfico de correlación con la variable objetivo
plt.figure(figsize=(8,6))
corr_target.drop('charges').plot(kind='bar', color='coral')
plt.title('Correlación de variables con Cargos')
plt.ylabel('Correlación')
plt.show()

# ======================================================
# 🔹 10. División del dataset en train y test (80/20)
# ======================================================
train, test = train_test_split(data_ready, test_size=0.2, random_state=42)
print("\nProporción media de cargos en train:", train['charges'].mean())
print("Proporción media de cargos en test:", test['charges'].mean())

# Guardar datasets
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)
print("\nDatasets guardados como 'train.csv' y 'test.csv'. Listos para usar en regresión.")